# Long Short Term Memory

In [1]:
import csv
import re
from abc import ABC, abstractmethod
import numpy as np

np.random.seed(99)

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def gradient(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.gradient()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def __add__(self, other):
        p = Tensor(self.data + other.data)

        def gradient_fn():
            self.grad = p.grad
            other.grad = p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def __mul__(self, other):
        p = Tensor(self.data * other.data)

        def gradient_fn():
            self.grad = p.grad * other.data
            other.grad = p.grad * self.data

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def concat(self, other, axis):
        p = Tensor(np.concatenate([self.data, other.data], axis=axis))

        def gradient_fn():
            self.grad, other.grad = np.split(p.grad, [self.data.shape[axis]], axis=axis)

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p


In [3]:
class Sequence:

    def __init__(self, tokens, vocabulary_size, batch_size):
        self.tokens = tokens
        self.vocabulary_size = vocabulary_size
        self.batch_size = batch_size

    def __len__(self):  # 3
        return len(self.tokens) - self.batch_size

    def __getitem__(self, index):  # 4
        return (Tensor([self.tokens[index:index + self.batch_size]]),
                Tensor([self.embedding(self.tokens[index + self.batch_size:
                                                   index + self.batch_size + 1])]))

    def embedding(self, index):
        ebd = np.zeros(self.vocabulary_size)
        ebd[index] = 1
        return ebd

In [4]:
class DataLoader:

    def __init__(self, sequence_batch_size):
        self.sequence_batch_size = sequence_batch_size

        self.reviews = []
        self.sentiments = []
        with open('reviews.csv', 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for r in self.reviews:
            split_reviews.append(self.clean_text(r.lower()).split())

        self.vocabulary = set(w for r in split_reviews for w in r)
        self.word2index = {w: idx for idx, w in enumerate(self.vocabulary)}
        self.index2word = {idx: w for idx, w in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[w] for w in r if w in self.word2index] for r in split_reviews]

        self.train()

    @staticmethod
    def clean_text(text):
        txt = re.sub(r'<[^>]+>', '', text)
        txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
        return txt

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def train(self):
        self.sequences = self.tokens[:-10]

    def eval(self):
        self.sequences = self.tokens[-10:]

    def __len__(self):  # 3
        return len(self.sequences)

    def __getitem__(self, index):  # 4
        return Sequence(self.sequences[index], len(self.vocabulary), self.sequence_batch_size)

In [5]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, *args):
        return self.forward(*args)

    @abstractmethod
    def forward(self, *args):
        pass

    def parameters(self):
        return []

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

In [6]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.in_size = in_size
        self.out_size = out_size

        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad.T @ x.data
            self.bias.grad = np.sum(p.grad, axis=0)
            x.grad = p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

In [7]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

In [8]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=1):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        self.weight = Tensor(np.random.rand(embedding_size, vocabulary_size) / vocabulary_size)

    def forward(self, x: Tensor):
        p = Tensor(np.sum(self.weight.data.T[x.data], axis=self.axis))

        def gradient_fn():
            if self.weight.grad is None:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    def parameters(self):
        return [self.weight]

In [9]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad = p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

In [10]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad = p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

In [11]:
class CELoss:

    def __call__(self, p: Tensor, y: Tensor):
        exp = np.exp(p.data - np.max(p.data, axis=-1, keepdims=True))
        softmax = exp / np.sum(exp, axis=-1, keepdims=True)

        log = np.log(softmax + 1e-10)
        ce = Tensor(0 - np.sum(y.data * log) / len(p.data))

        def gradient_fn():
            p.grad = (softmax - y.data) / len(p.data)

        ce.gradient_fn = gradient_fn
        ce.parents = {p}
        return ce

In [12]:
class SGD:

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

In [13]:
class LSTM(Layer):

    def __init__(self, vocabulary_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size

        self.embedding = Embedding(vocabulary_size, embedding_size)
        self.forget_gate = Linear(embedding_size * 2, embedding_size)
        self.input_gate = Linear(embedding_size * 2, embedding_size)
        self.output_gate = Linear(embedding_size * 2, embedding_size)
        self.cell_update = Linear(embedding_size * 2, embedding_size)
        self.output = Linear(embedding_size, vocabulary_size)
        self.sigmoid = Sigmoid()
        self.tanh = Tanh()

        self.layers = [self.embedding,
                       self.forget_gate,
                       self.input_gate,
                       self.output_gate,
                       self.cell_update,
                       self.output,
                       self.sigmoid,
                       self.tanh]

    def __call__(self, x: Tensor, c: Tensor, h: Tensor):
        return self.forward(x, c, h)

    def forward(self, x: Tensor, c: Tensor, h: Tensor):
        if not c:
            c = Tensor(np.zeros((1, self.embedding_size)))
        if not h:
            h = Tensor(np.zeros((1, self.embedding_size)))

        embedding_feature = self.embedding(x)
        concat_feature = embedding_feature.concat(h, axis=1)
        forget_hidden = self.sigmoid(self.forget_gate(concat_feature))
        input_hidden = self.sigmoid(self.input_gate(concat_feature))
        output_hidden = self.sigmoid(self.output_gate(concat_feature))
        cell_hidden = self.tanh(self.cell_update(concat_feature))
        cell_feature = forget_hidden * c + input_hidden * cell_hidden
        hidden_feature = output_hidden * self.tanh(cell_feature)

        return (self.output(hidden_feature),
                Tensor(cell_feature.data),
                Tensor(hidden_feature.data))

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

In [14]:
LEARNING_RATE = 0.02
BATCHES = 2
EPOCHS = 10

In [15]:
dataset = DataLoader(BATCHES)

model = LSTM(len(dataset.vocabulary), 64)

loss = CELoss()
sgd = SGD(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    for i in range(len(dataset)):
        sequence = dataset[i]

        cell = hidden = None
        for i in range(len(sequence)):
            feature, label = sequence[i]

            prediction, cell, hidden = model(feature, cell, hidden)
            error = loss(prediction, label)

            error.gradient()
            sgd.backward()

print(f'Prediction: {prediction.data}')
print(f'Error: {error.data}')

Prediction: [[-0.93798411  0.4911417  -0.11154155  0.51804935  0.06640075 -0.93580084
  -0.93444315 -0.72409664 -2.35024194 -0.08388275 -0.62186398 -0.06347789
  -0.83103121 -0.87093843 -0.67938979  0.44428378 -0.30342118 -0.51982411
  -0.76820929  0.50804689 -0.95645536  6.82384488  0.296484    2.57684255
  -0.72242256  1.3310878  -0.06362267  1.61691078 -0.64349568 -0.48945141
  -0.40067393  1.56438469 -0.26289259 -0.61335493  0.55970473 -0.73275055
  -0.21004898 -0.72222561 -0.91419266 -0.37993973 -1.37468615 -0.68916369
   0.75806775 -0.97652255 -0.73833584 -1.13049271 -0.8701522   1.95822624
   0.32927865  0.41530536  1.34331704 -0.23186723 -0.8236861   1.90097933
  -1.05986737 -0.59665934 -0.80641513  1.16610249  0.93550778 -0.07417397
   0.56053606 -0.95313499 -0.65142879  0.71224323  1.03372336  3.87205552
  -1.09908996 -2.00427331 -0.87998705  1.3758861   0.19029811 -0.68861757
  -1.08700392 -0.81801658  0.25517476 -1.51519516  0.75511229 -0.8291522
  -0.24689281 -0.10481102  

In [16]:
dataset.eval()

for i in range(len(dataset)):
    sequence = dataset[i]

    feature, label = sequence[0]
    original = sequence.tokens[:BATCHES]
    generated = original.copy()

    cell = hidden = None
    for j in range(len(sequence)):
        feature, label = sequence[j]

        prediction, cell, hidden = model(feature, cell, hidden)
        original.append(sequence.tokens[j + BATCHES])
        generated.append(prediction.data.argmax())

    print(f'original: {dataset.decode(original)}')
    print(f'generated: {dataset.decode(generated)}')

original: worst movie with awful music the actor did a boring job actress director character screenplay scene but or it at by
generated: worst movie the boring screenplay the screenplay and a poor job actor was screenplay screenplay scene scene in her in by
original: i recommend this film the actress was wonderful and the music was amazing actor director character scene plot by was
generated: i recommend this screenplay her and and wonderful i the actor was i i screenplay screenplay screenplay screenplay screenplay by
original: poor movie the story and actor were terrible i disappointed the director actress character action screenplay scene by is
generated: poor movie the actor and the the i i the the actor and screenplay screenplay screenplay scene scene was
original: excellent film i saw this time perfect performance by the actor and actress director screenplay scene character his her
generated: excellent film i this the time i performance screenplay the actress was were were screenp